## 01 Data_Excel-pd

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output

### 1. Loading Data

#### Load CSV

In [2]:
par = "par16"
file = "par16_com4_data_2021-04-01-09.40.51"
path = "../data/csv/round1/{file}.csv".format(file = file)

Load CSV and remove timestamps

#### 1.1 Visual 1s

In [3]:
df_visual_1 = pd.read_csv(path)
df_visual_1 = df_visual_1.drop(["timestamps"], axis=1)

#### 1.2 Visual 2s

In [4]:
df_visual_2 = pd.read_csv(path)
df_visual_2 = df_visual_2.drop(["timestamps"], axis=1)
# df_visual_1.head()

#### 1.3 Imagery 1s

In [5]:
df_imagery_1 = pd.read_csv(path)
df_imagery_1 = df_imagery_1.drop(["timestamps"], axis=1)
# df_imagery.head()

#### 1.4 Imagery 2s

In [6]:
df_imagery_2 = pd.read_csv(path)
df_imagery_2 = df_imagery_2.drop(["timestamps"], axis=1)
# df_imagery.head()

### 2. Checking Markers

Let's look at how the marker was generated.  Here is the format:

- [block, trial, label, time] <br>
label = 1,2,3,4,5,6,black <br>
time = 1s,2s
- [0] = warmup

In [7]:
print(df_visual_1['Marker'].unique())

['0' 'Training' '1,trial break,Fixation' '1,0,2,1' '1,task break,Fixation'
 '1,0,imagery,1' '1,0,4,2' '1,0,imagery,2' '1,0,6,2' '1,0,1,1' '1,0,3,1'
 '1,0,5,1' '1,1,2,2' '1,1,imagery,2' '1,1,4,1' '1,1,imagery,1' '1,1,6,1'
 '1,1,1,1' '1,1,3,2' '1,1,5,1' '1,2,2,2' '1,2,imagery,2' '1,2,4,2'
 '1,2,6,1' '1,2,imagery,1' '1,2,1,1' '1,2,3,2' '1,2,5,2' '1,3,2,2'
 '1,3,imagery,2' '1,3,4,1' '1,3,imagery,1' '1,3,6,1' '1,3,1,2' '1,3,3,1'
 '1,3,5,2' '1,4,2,1' '1,4,imagery,1' '1,4,4,1' '1,4,6,2' '1,4,imagery,2'
 '1,4,1,1' '1,4,3,2' '1,4,5,2' '2,trial break,Fixation' '2,0,2,2'
 '2,task break,Fixation' '2,0,imagery,2' '2,0,4,1' '2,0,imagery,1'
 '2,0,6,1' '2,0,1,1' '2,0,3,1' '2,0,5,2' '2,1,2,1' '2,1,imagery,1'
 '2,1,4,2' '2,1,imagery,2' '2,1,6,2' '2,1,1,1' '2,1,3,1' '2,1,5,2'
 '2,2,2,1' '2,2,imagery,1' '2,2,4,2' '2,2,imagery,2' '2,2,6,2' '2,2,1,1'
 '2,2,3,2' '2,2,5,2' '2,3,2,1' '2,3,imagery,1' '2,3,4,2' '2,3,imagery,2'
 '2,3,6,1' '2,3,1,2' '2,3,3,2' '2,3,5,2' '2,4,2,1' '2,4,imagery,1'
 '2,4,4,2' '2,4,ima

Since we set our marker to have 4 info: #block, #trial, label, time.  We gonna split and get the class for the markers.   **Note that we shall reserve 0 for no event for raw mne, thus we shall represent class 0-9 using label 1-10.**

#### 2.1 Visual 1 s

In [8]:
#use numpy as another view of the pandas columns for faster operation
marker_np_visual_1 = df_visual_1['Marker'].values
marker_np_visual_1 = marker_np_visual_1.astype(str)

### collecting indices of visual 1 s
visual_1_idx = []

for idx, marker in enumerate(marker_np_visual_1):
    if marker != '0':
        m = marker.split(",")
        if "Fixation" in marker:
            marker_np_visual_1[idx] = 0
        elif "trial break" in marker:
            marker_np_visual_1[idx] = 0
        elif "task break" in marker:
            marker_np_visual_1[idx] = 0
        elif "imagery" in marker: # remove black --> visual
            marker_np_visual_1[idx] = 0
        elif marker[-1] == '2':   # remove time = 2s --> 1s 
#             print('im HEREEEEEEEEEEEEEEEEEEEEE')
            marker_np_visual_1[idx] = 0
        elif "," in marker:
            marker_np_visual_1[idx] = m[2] # get classes
            visual_1_idx.append(idx)
        else:
            marker_np_visual_1[idx] = 0

# print(df_visual_1['Marker'].unique())
print(np.unique(marker_np_visual_1))
df_visual_1['Marker']= marker_np_visual_1

['0' '1' '2' '3' '4' '5' '6']


In [9]:
print(df_visual_1.groupby('Marker').nunique())

            F3      F8      F7      F4      P3      P4      T4      T3  \
Marker                                                                   
0       269933  235772  234907  277379  258376  263366  261066  256698   
1           39      39      39      39      39      39      39      39   
2           43      43      43      43      43      43      43      43   
3           38      38      38      38      38      38      38      38   
4           41      41      41      41      41      41      41      41   
5           31      31      31      31      31      31      31      31   
6           27      27      27      27      27      27      27      27   

           FP2     FP1      C4      C3      T6      T5      O1      O2  
Marker                                                                  
0       242262  243737  245081  232214  239539  279198  233597  252393  
1           39      39      39      39      39      39      39      39  
2           43      43      43      43   

In [10]:
print('the number of stim in total: ',len(visual_1_idx))
print(visual_1_idx)

the number of stim in total:  219
[7350, 9767, 10414, 11039, 12562, 13206, 13840, 15349, 17768, 18404, 21706, 22336, 23841, 25393, 26054, 27574, 31759, 32408, 33058, 33716, 35277, 37694, 38335, 39863, 42274, 44700, 46262, 49561, 53752, 57831, 59366, 65289, 66818, 68371, 69009, 69648, 71182, 75438, 77838, 78467, 79112, 81146, 81792, 83335, 83954, 84616, 86120, 86756, 87379, 88893, 90387, 91039, 91674, 92293, 93818, 94466, 95119, 95764, 99962, 111486, 114834, 116356, 118802, 119438, 120090, 120726, 122273, 124708, 125342, 126874, 127521, 131016, 133393, 134027, 135579, 138010, 138638, 140179, 142618, 144137, 145645, 147207, 147863, 149394, 151834, 157958, 158576, 159209, 159868, 160503, 162935, 164452, 165076, 175563, 177117, 178692, 180577, 181237, 183676, 187916, 189453, 190119, 191641, 194050, 194696, 197113, 197774, 198411, 199074, 201453, 202073, 202725, 204922, 207376, 208016, 209522, 210185, 211707, 214123, 214764, 217137, 218668, 219309, 220812, 224110, 224770, 226328, 226972, 22

#### 2.2 Imagery 1 s

In [11]:
marker_np_imagery_1 = df_imagery_1['Marker'].values
marker_np_imagery_1 = marker_np_imagery_1.astype(str)

### collecting indices of imagery 1 s (the values of these indices will need to be replaced by the values of visual indices)
imagery_1_idx = [] # rows we need

for idx, marker in enumerate(marker_np_imagery_1):
    if marker != '0':
        m = marker.split(",")
        if "imagery" in marker and marker[-1] == '1' : # get black and 1s
            imagery_1_idx.append(idx)

In [12]:
print('the number of stim in total: ',len(imagery_1_idx))
print(imagery_1_idx)

the number of stim in total:  219
[7604, 10028, 10653, 11307, 12820, 13456, 14092, 15609, 18019, 18638, 21950, 22582, 24098, 25667, 26300, 27828, 32022, 32675, 33328, 33982, 35543, 37950, 38594, 40133, 42511, 44961, 46518, 49802, 54004, 58098, 59638, 65531, 67076, 68623, 69261, 69923, 71448, 75674, 78082, 78725, 79356, 81406, 82049, 83569, 84227, 84855, 86370, 86992, 87636, 89126, 90656, 91287, 91906, 92558, 94081, 94734, 95376, 96023, 100199, 111758, 115076, 116623, 119054, 119706, 120341, 120971, 122529, 124957, 125592, 127136, 127772, 131248, 133641, 134293, 135848, 138249, 138902, 140446, 142853, 144392, 145910, 147476, 148120, 149655, 152072, 158190, 158823, 159479, 160116, 160751, 163166, 164691, 165327, 175828, 177393, 178948, 180854, 181513, 183949, 188170, 189731, 190380, 191892, 194309, 194949, 197389, 198024, 198689, 199326, 201689, 202339, 202997, 205191, 207630, 208260, 209798, 210454, 211953, 214376, 214996, 217371, 218923, 219559, 221056, 224384, 225048, 226588, 227205, 

### For example
df_imagery_1['Marker'][14141] = df_visual_1['Marker'].iloc[13886]

In [13]:
df_imagery_1['Marker'] = 0
for i in range(len(imagery_1_idx)):
    df_imagery_1['Marker'].iloc[imagery_1_idx[i]] = df_visual_1['Marker'].iloc[visual_1_idx[i]]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [14]:
print(df_imagery_1.groupby('Marker').nunique())

            F3      F8      F7      F4      P3      P4      T4      T3  \
Marker                                                                   
0       269948  235762  234915  277374  258370  263385  261066  256702   
1           39      39      39      39      39      39      39      39   
2           43      43      43      43      43      43      43      43   
3           38      38      38      38      38      38      38      38   
4           41      41      41      41      41      41      41      41   
5           31      31      31      31      31      31      31      31   
6           27      27      27      27      27      27      27      27   

           FP2     FP1      C4      C3      T6      T5      O1      O2  
Marker                                                                  
0       242248  243751  245086  232210  239528  279203  233591  252382  
1           39      39      39      39      39      39      39      39  
2           43      43      43      43   

#### 2.3 Visual 2 s

In [15]:
#use numpy as another view of the pandas columns for faster operation
marker_np_visual_2 = df_visual_2['Marker'].values
marker_np_visual_2 = marker_np_visual_2.astype(str)

### collecting indices of visual 2 s
visual_2_idx = []

for idx, marker in enumerate(marker_np_visual_2):
    if marker != '0':
        m = marker.split(",")
#         print(m[-1])
        if "Fixation" in marker:
            marker_np_visual_2[idx] = 0
        elif "trial break" in marker:
            marker_np_visual_2[idx] = 0
        elif "task break" in marker:
            marker_np_visual_2[idx] = 0
        elif "imagery" in marker: # remove black --> visual
            marker_np_visual_2[idx] = 0
        elif marker[-1] == '1':   # remove time = 1s --> 2s 
#             print('im HEREEEEEEEEEEEEEEEEEEEEE')
            marker_np_visual_2[idx] = 0
        elif "," in marker:
            marker_np_visual_2[idx] = m[2] # get classes
            visual_2_idx.append(idx)
        else:
            marker_np_visual_2[idx] = 0

# print(df_visual_1['Marker'].unique())
print(np.unique(marker_np_visual_2))
df_visual_2['Marker']= marker_np_visual_2

['0' '1' '2' '3' '4' '5' '6']


In [16]:
print('the number of stim in total: ',len(visual_2_idx))
print(visual_2_idx)

the number of stim in total:  231
[7987, 8892, 11693, 14478, 15994, 16887, 19022, 19936, 20833, 22967, 24483, 26688, 28212, 29079, 30882, 34368, 35926, 36799, 38979, 40519, 41411, 42897, 43791, 45346, 46904, 47778, 48649, 50190, 51098, 51968, 52873, 56042, 56926, 58486, 60022, 60894, 61764, 62640, 63536, 64409, 65916, 67459, 70307, 71833, 72717, 73611, 74521, 76061, 76955, 82436, 85243, 88021, 89512, 92944, 96408, 97286, 98194, 99074, 100585, 101469, 102372, 105230, 106131, 107015, 107921, 108790, 109690, 110599, 112145, 113044, 113959, 115461, 117005, 117916, 121359, 122914, 123810, 125978, 128155, 131631, 132515, 134680, 136232, 137123, 139286, 140832, 141722, 143237, 144778, 146296, 148508, 150041, 150934, 152458, 153355, 155288, 156190, 157090, 161137, 162023, 163554, 165711, 166625, 167525, 168440, 169314, 170211, 171122, 171990, 172884, 173791, 174690, 176216, 177777, 181899, 182789, 184336, 185245, 186122, 187021, 188553, 190766, 192279, 193173, 195335, 196204, 199712, 200581, 2

In [17]:
print(df_visual_2.groupby('Marker').nunique())

            F3      F8      F7      F4      P3      P4      T4      T3  \
Marker                                                                   
0       269934  235771  234902  277360  258369  263359  261060  256680   
1           36      36      36      36      36      36      36      36   
2           32      32      32      32      32      32      32      32   
3           37      37      37      37      37      37      37      37   
4           34      34      34      34      34      34      34      34   
5           44      44      44      44      44      44      44      44   
6           48      48      48      48      48      48      48      48   

           FP2     FP1      C4      C3      T6      T5      O1      O2  
Marker                                                                  
0       242241  243728  245084  232205  239526  279190  233591  252361  
1           36      36      36      36      36      36      36      36  
2           32      32      32      32   

#### 2.4 Imagery 2 s

In [18]:
marker_np_imagery_2 = df_imagery_2['Marker'].values
marker_np_imagery_2 = marker_np_imagery_2.astype(str)

### collecting indices of imagery 2 s (the values of these indices will need to be replaced by the values of visual indices)
imagery_2_idx = []
for idx, marker in enumerate(marker_np_imagery_2):
    if marker != '0':
        m = marker.split(",")
        if "imagery" in marker and marker[-1] == '2' : # get black and 2s
            imagery_2_idx.append(idx)


In [19]:
print('the number of stim in total: ',len(imagery_2_idx))
print(imagery_2_idx)

the number of stim in total:  231
[8384, 9258, 12055, 14839, 16377, 17256, 19425, 20323, 21195, 23330, 24881, 27063, 28569, 29478, 31248, 34767, 36290, 37186, 39352, 40901, 41766, 43280, 44189, 45751, 47265, 48140, 49049, 50589, 51456, 52364, 53243, 56415, 57322, 58855, 60383, 61253, 62129, 63026, 63898, 64775, 66308, 67860, 70671, 72204, 73098, 74005, 74925, 76445, 77327, 82824, 85606, 88384, 89877, 93307, 96777, 97683, 98562, 99453, 100960, 101861, 102772, 105620, 106504, 107411, 108282, 109182, 110088, 110977, 112533, 113448, 114326, 115843, 117406, 118292, 121762, 123300, 124197, 126366, 128541, 132005, 132880, 135068, 136611, 137500, 139668, 141212, 142108, 143626, 145134, 146695, 148884, 150423, 151325, 152844, 153742, 155680, 156581, 157448, 161513, 162424, 163942, 166115, 167014, 167927, 168804, 169698, 170611, 171479, 172373, 173280, 174180, 175053, 176607, 178179, 182278, 183164, 184733, 185611, 186514, 187405, 188943, 191131, 192666, 193539, 195691, 196601, 200070, 200944, 2

In [20]:
df_imagery_2['Marker'] = '0'
for i in range(len(imagery_2_idx)):
    df_imagery_2['Marker'].iloc[imagery_2_idx[i]] = df_visual_2['Marker'].iloc[visual_2_idx[i]]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [21]:
print(df_imagery_2.groupby('Marker').nunique())

            F3      F8      F7      F4      P3      P4      T4      T3  \
Marker                                                                   
0       269923  235755  234924  277360  258363  263359  261068  256688   
1           36      36      36      36      36      36      36      36   
2           32      32      32      32      32      32      32      32   
3           37      37      37      37      37      37      37      37   
4           34      34      34      34      34      34      34      34   
5           44      44      44      44      44      44      44      44   
6           48      48      48      48      48      48      48      48   

           FP2     FP1      C4      C3      T6      T5      O1      O2  
Marker                                                                  
0       242240  243723  245072  232209  239526  279208  233592  252378  
1           36      36      36      36      36      36      36      36  
2           32      32      32      32   

### 3. Save Data

In [22]:
df_visual_1.to_pickle("data/pd/{par}/{file}_visual_1.pkl".format(par=par, file=file))
df_imagery_1.to_pickle("data/pd/round1/{par}/{file}_imagery_1.pkl".format(par=par, file=file))
df_visual_2.to_pickle("data/pd/round1/{par}/{file}_visual_2.pkl".format(par=par, file=file))
df_imagery_2.to_pickle("data/pd/round1/{par}/{file}_imagery_2.pkl".format(par=par, file=file))